In [33]:
import pandas as pd
import numpy as np

In [34]:
from pyspark.sql import SparkSession
from pyspark.sql.types import *
import seaborn as sns
import matplotlib.pyplot as plt

In [35]:
spark = SparkSession.builder.master('local[4]').appName('ml').getOrCreate()

# Dataset - 1(bank-full.csv)

In [36]:
data = spark.read.csv('bank-full.csv',sep = ";",header=True,inferSchema=True)

In [37]:
data.count()

45211

### Converting to Pandas

In [38]:
pdf=data.toPandas()

#### to display maximum no. of columns

In [39]:
pd.pandas.set_option('display.max_columns',None)

In [40]:
pdf.head(5)

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,58,management,married,tertiary,no,2143,yes,no,unknown,5,may,261,1,-1,0,unknown,no
1,44,technician,single,secondary,no,29,yes,no,unknown,5,may,151,1,-1,0,unknown,no
2,33,entrepreneur,married,secondary,no,2,yes,yes,unknown,5,may,76,1,-1,0,unknown,no
3,47,blue-collar,married,unknown,no,1506,yes,no,unknown,5,may,92,1,-1,0,unknown,no
4,33,unknown,single,unknown,no,1,no,no,unknown,5,may,198,1,-1,0,unknown,no


### Dropping balance and day column

In [41]:
pdf = pdf.drop(["balance","day"], axis=1)

In [42]:
pdf.head(5)

,age,job,marital,education,default,housing,loan,contact,month,duration,campaign,pdays,previous,poutcome,y
0,58,management,married,tertiary,no,yes,no,unknown,may,261,1,-1,0,unknown,no
1,44,technician,single,secondary,no,yes,no,unknown,may,151,1,-1,0,unknown,no
2,33,entrepreneur,married,secondary,no,yes,yes,unknown,may,76,1,-1,0,unknown,no
3,47,blue-collar,married,unknown,no,yes,no,unknown,may,92,1,-1,0,unknown,no
4,33,unknown,single,unknown,no,no,no,unknown,may,198,1,-1,0,unknown,no


### renamed column for unform naming before concat operation

In [50]:
new_cols = ["emp_var_rate", "cons_price_idx", "cons_conf_idx", "euribor_3m", "nr_employed"]
col_no = 14
for column in new_cols:
    pdf.insert(col_no, column, value = np.nan)
    col_no = col_no + 1

In [51]:
pdf.head()

,age,job,marital,education,default,housing,loan,contact,month,duration,campaign,pdays,previous,poutcome,emp_var_rate,cons_price_idx,cons_conf_idx,euribor_3m,nr_employed,y
0,58,management,married,tertiary,no,yes,no,unknown,may,261,1,-1,0,unknown,NaN,NaN,NaN,NaN,NaN,no
1,44,technician,single,secondary,no,yes,no,unknown,may,151,1,-1,0,unknown,NaN,NaN,NaN,NaN,NaN,no
2,33,entrepreneur,married,secondary,no,yes,yes,unknown,may,76,1,-1,0,unknown,NaN,NaN,NaN,NaN,NaN,no
3,47,blue-collar,married,unknown,no,yes,no,unknown,may,92,1,-1,0,unknown,NaN,NaN,NaN,NaN,NaN,no
4,33,unknown,single,unknown,no,no,no,unknown,may,198,1,-1,0,unknown,NaN,NaN,NaN,NaN,NaN,no


## Adding year mapper function to add year in dataset - 1(bank - full.csv)

In [94]:
def year_mapper(data, start_yr):
    month_lst = ["jan", "feb", "mar", "apr", "may", "jun", "jul", "aug", "sep", "oct", "nov", "dec"]

    # Make a copy of the original dataframe
    new_data = data.copy()

    # Insert a new "year" column filled with zeros
    new_data.insert(loc=0, column="year", value=0)

    # Set the first year to the start year
    current_year = int(start_yr)
    new_data.at[0, "year"] = current_year

    # Loop through the rows of the dataframe, updating the year column when the month changes
    for i in range(1, len(new_data)):
        # If the current month is earlier in the year than the previous month, increment the year
        if month_lst.index(new_data["month"][i]) < month_lst.index(new_data["month"][i-1]):
            current_year += 1

        new_data.at[i, "year"] = current_year

        # If the current year exceeds the end year, break out of the loop
        if current_year > 2010:
            break

    return new_data

In [96]:
result_bankfull = year_mapper( data = pdf, start_yr = 2008)
result_bankfull.head()

,year,age,job,marital,education,default,housing,loan,contact,month,duration,campaign,pdays,previous,poutcome,emp_var_rate,cons_price_idx,cons_conf_idx,euribor_3m,nr_employed,y
0,2008,58,management,married,tertiary,no,yes,no,unknown,may,261,1,-1,0,unknown,NaN,NaN,NaN,NaN,NaN,no
1,2008,44,technician,single,secondary,no,yes,no,unknown,may,151,1,-1,0,unknown,NaN,NaN,NaN,NaN,NaN,no
2,2008,33,entrepreneur,married,secondary,no,yes,yes,unknown,may,76,1,-1,0,unknown,NaN,NaN,NaN,NaN,NaN,no
3,2008,47,blue-collar,married,unknown,no,yes,no,unknown,may,92,1,-1,0,unknown,NaN,NaN,NaN,NaN,NaN,no
4,2008,33,unknown,single,unknown,no,no,no,unknown,may,198,1,-1,0,unknown,NaN,NaN,NaN,NaN,NaN,no


In [103]:
result_bankfull["year"].value_counts()

2008    27729
2009    14862
2010     2620
Name: year, dtype: int64

### Mapper function to insert indexes value into columns 

In [109]:
def map_index(data):
    
    index_2008 = {"may":{"emp_var_rate":1.1, "cons_price_idx":93.994, "cons_conf_idx":-36.4, "euribor_3m":4.85, "nr_employed":5191},
             "jun":{"emp_var_rate":1.4, "cons_price_idx":94.465, "cons_conf_idx":-41.8, "euribor_3m":4.86, "nr_employed":5228.1},
             "jul":{"emp_var_rate":1.4, "cons_price_idx":93.918, "cons_conf_idx":-42.7, "euribor_3m":4.96, "nr_employed":5228.1},
             "aug":{"emp_var_rate":1.4, "cons_price_idx":93.444, "cons_conf_idx":-36.1, "euribor_3m":4.965, "nr_employed":5228.1},
             "oct":{"emp_var_rate":-0.1, "cons_price_idx":93.798, "cons_conf_idx":-40.4, "euribor_3m":5, "nr_employed":5195.8},
             "nov":{"emp_var_rate":-0.1, "cons_price_idx":93.2, "cons_conf_idx":-42, "euribor_3m":4.406, "nr_employed":5195.8},
             "dec":{"emp_var_rate":-0.2, "cons_price_idx":92.75, "cons_conf_idx":-45.9, "euribor_3m":3.563, "nr_employed":5176.3}}
    
    index_2009 = {"mar":{"emp_var_rate":-1.8, "cons_price_idx":92.84, "cons_conf_idx":-50, "euribor_3m":1.811, "nr_employed":5099.1},
              "apr":{"emp_var_rate":-1.8, "cons_price_idx":93.075, "cons_conf_idx":-47.1, "euribor_3m":1.498, "nr_employed":5099.1},
              "may":{"emp_var_rate":-1.8, "cons_price_idx":92.89, "cons_conf_idx":-46.2, "euribor_3m":1.334, "nr_employed":5099.1},
             "jun":{"emp_var_rate":-2.9, "cons_price_idx":92.963, "cons_conf_idx":-40.8, "euribor_3m":1.26, "nr_employed":5076.2},
             "jul":{"emp_var_rate":-2.9, "cons_price_idx":93.469, "cons_conf_idx":-33.6, "euribor_3m":1.072, "nr_employed":5076.2},
             "aug":{"emp_var_rate":-2.9, "cons_price_idx":92.201, "cons_conf_idx":-31.4, "euribor_3m":0.884, "nr_employed":5076.2},
             "sep":{"emp_var_rate":-3.4, "cons_price_idx":92.379, "cons_conf_idx":-29.8, "euribor_3m":0.813, "nr_employed":5017.5},
             "oct":{"emp_var_rate":-3.4, "cons_price_idx":92.431, "cons_conf_idx":-26.9, "euribor_3m":0.754, "nr_employed":5017.5},
             "nov":{"emp_var_rate":-3.4, "cons_price_idx":92.649, "cons_conf_idx":-30.1, "euribor_3m":0.722, "nr_employed":5017.5},
             "dec":{"emp_var_rate":-3, "cons_price_idx":92.713, "cons_conf_idx":-33, "euribor_3m":0.718, "nr_employed":5023.5}}
    
    index_2010 = {"mar":{"emp_var_rate":--1.8, "cons_price_idx":92.369, "cons_conf_idx":-34.8, "euribor_3m":0.655, "nr_employed":5008.7},
              "apr":{"emp_var_rate":-1.8, "cons_price_idx":93.749, "cons_conf_idx":-34.6, "euribor_3m":0.64, "nr_employed":5008.7},
              "may":{"emp_var_rate":-1.8, "cons_price_idx":93.876, "cons_conf_idx":-40, "euribor_3m":0.668, "nr_employed":5008.7},
             "jun":{"emp_var_rate":-1.7, "cons_price_idx":94.055, "cons_conf_idx":-39.8, "euribor_3m":0.704, "nr_employed":4991.6},
             "jul":{"emp_var_rate":-1.7, "cons_price_idx":94.215, "cons_conf_idx":-40.3, "euribor_3m":0.79, "nr_employed":4991.6},
             "aug":{"emp_var_rate":-1.7, "cons_price_idx":94.027, "cons_conf_idx":-38.3, "euribor_3m":0.898, "nr_employed":4991.6},
             "sep":{"emp_var_rate":-1.1, "cons_price_idx":94.199, "cons_conf_idx":-37.5, "euribor_3m":0.886, "nr_employed":4963.6},
             "oct":{"emp_var_rate":-1.1, "cons_price_idx":94.601, "cons_conf_idx":-49.5, "euribor_3m":0.959, "nr_employed":4963.6},
             "nov":{"emp_var_rate":-1.1, "cons_price_idx":94.767, "cons_conf_idx":-50.8, "euribor_3m":1.05, "nr_employed":4963.6}}
    
    new_data = data.copy()
    indx = [index_2008, index_2009, index_2010]
    years = [2008, 2009, 2010]

    for i in range(0 , len(years)):
        for months, indexes in indx[i].items():
            for index, index_val in indexes.items():
                new_data.loc[(new_data['year'] == years[i]) & (new_data['month'] == months), index] = index_val
    return new_data            

In [110]:
bank_full_index = map_index(data = result_bankfull)
bank_full_index.head()

,year,age,job,marital,education,default,housing,loan,contact,month,duration,campaign,pdays,previous,poutcome,emp_var_rate,cons_price_idx,cons_conf_idx,euribor_3m,nr_employed,y
0,2008,58,management,married,tertiary,no,yes,no,unknown,may,261,1,-1,0,unknown,1.1,93.994,-36.4,4.85,5191.0,no
1,2008,44,technician,single,secondary,no,yes,no,unknown,may,151,1,-1,0,unknown,1.1,93.994,-36.4,4.85,5191.0,no
2,2008,33,entrepreneur,married,secondary,no,yes,yes,unknown,may,76,1,-1,0,unknown,1.1,93.994,-36.4,4.85,5191.0,no
3,2008,47,blue-collar,married,unknown,no,yes,no,unknown,may,92,1,-1,0,unknown,1.1,93.994,-36.4,4.85,5191.0,no
4,2008,33,unknown,single,unknown,no,no,no,unknown,may,198,1,-1,0,unknown,1.1,93.994,-36.4,4.85,5191.0,no


In [111]:
bank_full_index.count()

year              45211
age               45211
job               45211
marital           45211
education         45211
default           45211
housing           45211
loan              45211
contact           45211
month             45211
duration          45211
campaign          45211
pdays             45211
previous          45211
poutcome          45211
emp_var_rate      41159
cons_price_idx    41159
cons_conf_idx     41159
euribor_3m        41159
nr_employed       41159
y                 45211
dtype: int64

# Dataset - 2 (additional-full.csv)

In [52]:
dataset_2 = spark.read.csv('bank-additional-full.csv',sep = ";",header=True,inferSchema=True)

In [54]:
dataset_2.count()

41188

### Converting dataset to pandas

In [55]:
pdf_2 = dataset_2.toPandas()

In [56]:
pdf_2.head(5)

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,duration,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,56,housemaid,married,basic.4y,no,no,no,telephone,may,mon,261,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
1,57,services,married,high.school,unknown,no,no,telephone,may,mon,149,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
2,37,services,married,high.school,no,yes,no,telephone,may,mon,226,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
3,40,admin.,married,basic.6y,no,no,no,telephone,may,mon,151,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
4,56,services,married,high.school,no,no,yes,telephone,may,mon,307,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no


### Replacing pdays 999 value to -1

In [57]:
pdf_2["pdays"] = pdf_2["pdays"].replace(999, -1)

In [59]:
pdf_2.head(5)

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,duration,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,56,housemaid,married,basic.4y,no,no,no,telephone,may,mon,261,1,-1,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
1,57,services,married,high.school,unknown,no,no,telephone,may,mon,149,1,-1,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
2,37,services,married,high.school,no,yes,no,telephone,may,mon,226,1,-1,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
3,40,admin.,married,basic.6y,no,no,no,telephone,may,mon,151,1,-1,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
4,56,services,married,high.school,no,no,yes,telephone,may,mon,307,1,-1,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no


### Dropping day_of_week column for now

In [61]:
pdf_2 = pdf_2.drop("day_of_week", axis=1)

In [62]:
pdf_2.head(5)

,age,job,marital,education,default,housing,loan,contact,month,duration,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,56,housemaid,married,basic.4y,no,no,no,telephone,may,261,1,-1,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
1,57,services,married,high.school,unknown,no,no,telephone,may,149,1,-1,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
2,37,services,married,high.school,no,yes,no,telephone,may,226,1,-1,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
3,40,admin.,married,basic.6y,no,no,no,telephone,may,151,1,-1,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
4,56,services,married,high.school,no,no,yes,telephone,may,307,1,-1,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no


### Renaming column names

In [66]:
old_col_list = ["emp.var.rate", "cons.price.idx", "cons.conf.idx", "euribor3m", "nr.employed"]
for i in range(0, len(old_col_list)):
    pdf_2.rename(columns={old_col_list[i]: new_cols[i]}, inplace=True)


In [67]:
pdf_2.head(5)

,age,job,marital,education,default,housing,loan,contact,month,duration,campaign,pdays,previous,poutcome,emp_var_rate,cons_price_idx,cons_conf_idx,euribor_3m,nr_employed,y
0,56,housemaid,married,basic.4y,no,no,no,telephone,may,261,1,-1,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
1,57,services,married,high.school,unknown,no,no,telephone,may,149,1,-1,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
2,37,services,married,high.school,no,yes,no,telephone,may,226,1,-1,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
3,40,admin.,married,basic.6y,no,no,no,telephone,may,151,1,-1,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
4,56,services,married,high.school,no,no,yes,telephone,may,307,1,-1,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no


## Adding year mapper function to add year to dataset - 2(addition-full.csv)

In [97]:
def year_mapper(data, start_yr):
    month_lst = ["jan", "feb", "mar", "apr", "may", "jun", "jul", "aug", "sep", "oct", "nov", "dec"]

    # Make a copy of the original dataframe
    new_data = data.copy()

    # Insert a new "year" column filled with zeros
    new_data.insert(loc=0, column="year", value=0)

    # Set the first year to the start year
    current_year = int(start_yr)
    new_data.at[0, "year"] = current_year

    # Loop through the rows of the dataframe, updating the year column when the month changes
    for i in range(1, len(new_data)):
        # If the current month is earlier in the year than the previous month, increment the year
        if month_lst.index(new_data["month"][i]) < month_lst.index(new_data["month"][i-1]):
            current_year += 1

        new_data.at[i, "year"] = current_year

        # If the current year exceeds the end year, break out of the loop
        if current_year > 2010:
            break

    return new_data

In [98]:
result_addition = year_mapper( data = pdf_2, start_yr = 2008)
result_addition.head()

,year,age,job,marital,education,default,housing,loan,contact,month,duration,campaign,pdays,previous,poutcome,emp_var_rate,cons_price_idx,cons_conf_idx,euribor_3m,nr_employed,y
0,2008,56,housemaid,married,basic.4y,no,no,no,telephone,may,261,1,-1,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
1,2008,57,services,married,high.school,unknown,no,no,telephone,may,149,1,-1,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
2,2008,37,services,married,high.school,no,yes,no,telephone,may,226,1,-1,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
3,2008,40,admin.,married,basic.6y,no,no,no,telephone,may,151,1,-1,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
4,2008,56,services,married,high.school,no,no,yes,telephone,may,307,1,-1,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no


In [104]:
result_addition["year"].value_counts()

2008    27690
2009    11440
2010     2058
Name: year, dtype: int64

## Concatinating two datasets into one

In [99]:
frames  = [result_bankfull, result_addition]

new_dataframe = pd.concat(frames)